In [6]:
# Imports
from ncclient import manager
from xmltodict import parse, unparse
from lxml.etree import fromstring
from pprint import pprint as pp

In [2]:
# Constants
DEVICE = {
    'host': '10.10.20.58',
    'port': 830,
    'username': 'admin',
    'password': 'Cisco123',
    'hostkey_verify': False,
    'device_params': {
        'name': 'nexus'
    }
}

In [14]:
# Get NETCONF Capabilities
with manager.connect(**DEVICE) as m:
    caps = m.server_capabilities

In [16]:
# Print NETCONF Capabilities
for c in caps:
    print(c.split('?')[0])

urn:ietf:params:netconf:base:1.0
urn:ietf:params:netconf:base:1.1
urn:ietf:params:netconf:capability:writable-running:1.0
urn:ietf:params:netconf:capability:rollback-on-error:1.0
urn:ietf:params:netconf:capability:candidate:1.0
urn:ietf:params:netconf:capability:validate:1.1
urn:ietf:params:netconf:capability:confirmed-commit:1.1
urn:ietf:params:netconf:capability:notification:1.0
urn:ietf:params:netconf:capability:interleave:1.0
urn:ietf:params:netconf:capability:with-defaults:1.0
http://cisco.com/ns/yang/cisco-nx-os-device
http://openconfig.net/yang/acl
http://openconfig.net/yang/bfd
http://openconfig.net/yang/bgp-policy
http://openconfig.net/yang/interfaces
http://openconfig.net/yang/interfaces/aggregate
http://openconfig.net/yang/interfaces/ethernet
http://openconfig.net/yang/interfaces/ip
http://openconfig.net/yang/interfaces/ip-ext
http://openconfig.net/yang/lacp
http://openconfig.net/yang/lldp
http://openconfig.net/yang/network-instance
http://openconfig.net/yang/network-instanc

In [65]:
# Construct filter to get serial number
filter = '''
<filter>
    <System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
        <serial/>
    </System>
</filter>
'''

In [66]:
# Get the serial number
with manager.connect(**DEVICE) as m:
    r = m.get(filter=filter)

In [77]:
print(r.data_ele.find('.//{http://cisco.com/ns/yang/cisco-nx-os-device}serial').text)

91OMZP37WBR


In [88]:
config = '''
<config>
    <System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
        <intf-items>
            <lb-items>
                <LbRtdIf-list>
                    <id>lo10</id>
                    <descr>Test NETCONF LB</descr>
                    <adminSt>up</adminSt>
                </LbRtdIf-list>
            </lb-items>
        </intf-items>
        <ipv4-items>
            <inst-items>
                <dom-items>
                    <Dom-list>
                        <name>default</name>
                        <if-items>
                            <If-list>
                                <id>lo10</id>
                                <addr-items>
                                    <Addr-list>
                                        <addr>1.1.1.2/32</addr>
                                    </Addr-list>
                                </addr-items>
                            </If-list>
                        </if-items>
                    </Dom-list>
                </dom-items>
            </inst-items>
        </ipv4-items>
    </System>
</config>
'''
fromstring(config)

<Element config at 0x7ff0ec2a1140>

In [89]:
# Create the interface
with manager.connect(**DEVICE) as m:
    r = m.edit_config(target='running', config=config)

In [90]:
# Display the response
print(r)

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns:nxos="http://www.cisco.com/nxos:1.0" xmlns:if="http://www.cisco.com/nxos:1.0:if_manager" xmlns:nfcli="http://www.cisco.com/nxos:1.0:nfcli" xmlns:vlan_mgr_cli="http://www.cisco.com/nxos:1.0:vlan_mgr_cli" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:45eb0848-9fc3-4969-9964-45938cc67b69">
    <ok/>
</rpc-reply>



In [175]:
# Create the BGP payload
config = '''
<config>
    <System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
        <bgp-items>
            <inst-items>
                <asn>65535</asn>
                <dom-items>
                    <Dom-list>
                        <name>default</name>
                        <rtrId>1.1.1.2</rtrId>
                    </Dom-list>
                </dom-items>
            </inst-items>
        </bgp-items>
    </System>
</config>
'''
fromstring(config)

<Element config at 0x7ff0ec1076c0>

In [176]:
# Configure BGP
with manager.connect(**DEVICE) as m:
    r = m.edit_config(target='running', config=config)
    print(r)

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns:nxos="http://www.cisco.com/nxos:1.0" xmlns:if="http://www.cisco.com/nxos:1.0:if_manager" xmlns:nfcli="http://www.cisco.com/nxos:1.0:nfcli" xmlns:vlan_mgr_cli="http://www.cisco.com/nxos:1.0:vlan_mgr_cli" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:d82bb6c2-0027-4876-86c4-386dfa31ed27">
    <ok/>
</rpc-reply>



In [139]:
# Add BGP prefix
config = '''
<config>
    <System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
        <bgp-items>
            <inst-items>
                <dom-items>
                    <Dom-list>
                        <name>default</name>
                        <af-items>
                            <DomAf-list>
                                <type>ipv4-ucast</type>
                                <prefix-items>
                                    <AdvPrefix-list>
                                        <addr>1.1.1.2/32</addr>
                                    </AdvPrefix-list>
                                </prefix-items>
                            </DomAf-list>
                        </af-items>
                    </Dom-list>
                </dom-items>
            </inst-items>
        </bgp-items>
    </System>
</config>
'''
fromstring(config)

<Element config at 0x7ff0ec1b6900>

In [141]:
# Send the BGP prefix config
with manager.connect(**DEVICE) as m:
    r = m.edit_config(target='running', config=config)

In [121]:
# Display the response
print(r)

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns:nxos="http://www.cisco.com/nxos:1.0" xmlns:if="http://www.cisco.com/nxos:1.0:if_manager" xmlns:nfcli="http://www.cisco.com/nxos:1.0:nfcli" xmlns:vlan_mgr_cli="http://www.cisco.com/nxos:1.0:vlan_mgr_cli" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:9e871231-1928-4990-8c93-e05f8312dc7a">
    <ok/>
</rpc-reply>



In [122]:
# Get the BGP prefix
filter = '''
<filter>
    <System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
        <bgp-items>
            <inst-items>
                <dom-items>
                    <Dom-list>
                        <name>default</name>
                        <af-items>
                            <DomAf-list>
                                <type>ipv4-ucast</type>
                                <prefix-items>
                                    <AdvPrefix-list/>
                                </prefix-items>
                            </DomAf-list>
                        </af-items>
                    </Dom-list>
                </dom-items>
            </inst-items>
        </bgp-items>
    </System>
</filter>
'''
fromstring(filter)

<Element filter at 0x7ff0ec250240>

In [124]:
# Get the serial number
with manager.connect(**DEVICE) as m:
    r = m.get_config(source='running', filter=filter)

print(r)

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns:nxos="http://www.cisco.com/nxos:1.0" xmlns:if="http://www.cisco.com/nxos:1.0:if_manager" xmlns:nfcli="http://www.cisco.com/nxos:1.0:nfcli" xmlns:vlan_mgr_cli="http://www.cisco.com/nxos:1.0:vlan_mgr_cli" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:c6ab2bf6-68ab-453e-9bf6-d1e0122c8a66">
    <data>
        <System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
            <bgp-items>
                <inst-items>
                    <dom-items>
                        <Dom-list>
                            <name>default</name>
                            <af-items>
                                <DomAf-list>
                                    <type>ipv4-ucast</type>
                                    <prefix-items>
                                        <AdvPrefix-list>
                                            <addr>1.1.1.2/32</addr>
                                        </AdvPrefix-list>
      

In [172]:
# Delete BGP prefix
# Add the following attribute to the <prefix-items> tag:
# xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0" nc:operation="delete"

# This abbreviated version also works:
# operation="delete"
config = '''
<config>
    <System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
        <bgp-items>
            <inst-items>
                <dom-items>
                    <Dom-list>
                        <name>default</name>
                        <af-items>
                            <DomAf-list>
                                <type>ipv4-ucast</type>
                                <prefix-items xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0" nc:operation="delete">
                                    <AdvPrefix-list>
                                        <addr>1.1.1.2/32</addr>
                                    </AdvPrefix-list>
                                </prefix-items>
                            </DomAf-list>
                        </af-items>
                    </Dom-list>
                </dom-items>
            </inst-items>
        </bgp-items>
    </System>
</config>
'''
fromstring(config)

<Element config at 0x7ff0ec11b200>

In [143]:
# Send the BGP prefix config
with manager.connect(**DEVICE) as m:
    r = m.edit_config(target='running', config=config)

In [144]:
# Display the response
print(r)

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns:nxos="http://www.cisco.com/nxos:1.0" xmlns:if="http://www.cisco.com/nxos:1.0:if_manager" xmlns:nfcli="http://www.cisco.com/nxos:1.0:nfcli" xmlns:vlan_mgr_cli="http://www.cisco.com/nxos:1.0:vlan_mgr_cli" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:de3b803a-93a8-4a7c-aac5-11a1458e6843">
    <ok/>
</rpc-reply>



---
---

In [162]:
# Create OpenConfig payload to get loopback interfaces
filter = '''
<filter>
    <interfaces xmlns="http://openconfig.net/yang/interfaces">
        <interface>
            <name>lo10</name>
            <config/>
            <subinterfaces>
                <subinterface>
                    <index/>
                    <ipv4>
                        <addresses>
                            <address>
                                <ip/>
                                <config/>
                            </address>
                        </addresses>
                    </ipv4>
                </subinterface>
            </subinterfaces>
        </interface>
    </interfaces>
</filter>
'''
fromstring(config)

<Element config at 0x7ff0df64c6c0>

In [163]:
# Send the payload to get interface details
with manager.connect(**DEVICE) as m:
    r = m.get_config(source='running', filter=filter)

print(r)

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns:nxos="http://www.cisco.com/nxos:1.0" xmlns:if="http://www.cisco.com/nxos:1.0:if_manager" xmlns:nfcli="http://www.cisco.com/nxos:1.0:nfcli" xmlns:vlan_mgr_cli="http://www.cisco.com/nxos:1.0:vlan_mgr_cli" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:deb9e3f1-e464-48b2-96c6-1be431c1af15">
    <data>
        <interfaces xmlns="http://openconfig.net/yang/interfaces">
            <interface>
                <name>lo10</name>
                <config>
                    <description>Test NETCONF LB</description>
                    <enabled>true</enabled>
                    <name>lo10</name>
                    <type xmlns:ianaift="urn:ietf:params:xml:ns:yang:iana-if-type">ianaift:softwareLoopback</type>
                </config>
                <subinterfaces>
                    <subinterface>
                        <index>0</index>
                        <ipv4 xmlns="http://openconfig.net/yang/interfaces/ip"

In [180]:
# Create OpenConfig payload to add a loopback interface
config = '''
<config>
    <interfaces xmlns="http://openconfig.net/yang/interfaces">
        <interface>
            <name>lo12</name>
            <config>
                <name>lo12</name>
                <enabled>true</enabled>
                <type xmlns:ianaift="urn:ietf:params:xml:ns:yang:iana-if-type">ianaift:softwareLoopback</type>
                <description>OpenConfig Loopback</description>
            </config>
            <subinterfaces>
                <subinterface>
                    <index>0</index>
                    <ipv4>
                        <addresses>
                            <address>
                                <ip>1.1.1.12</ip>
                                <config>
                                    <ip>1.1.1.12</ip>
                                    <prefix-length>32</prefix-length>
                                </config>
                            </address>
                        </addresses>
                    </ipv4>
                </subinterface>
            </subinterfaces>
        </interface>
    </interfaces>
</config>
'''

fromstring(config)

<Element config at 0x7ff0ec1451c0>

In [181]:
# Send the configuration payload
with manager.connect(**DEVICE) as m:
    r = m.edit_config(target='running', config=config)

    print(r)

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns:nxos="http://www.cisco.com/nxos:1.0" xmlns:if="http://www.cisco.com/nxos:1.0:if_manager" xmlns:nfcli="http://www.cisco.com/nxos:1.0:nfcli" xmlns:vlan_mgr_cli="http://www.cisco.com/nxos:1.0:vlan_mgr_cli" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:3817c10c-0052-45a4-847f-0d912bf87727">
    <ok/>
</rpc-reply>

